In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("../data/data_imputed_3.csv")

In [20]:
df

,timestamp_local,temp,aqi,co,no2,o3,pm10,pm25,so2
0,01/02/2022 0:00,12.6,385,1339.8,76.0,10.7,491.7,347.67,238.0
1,01/02/2022 1:00,11.5,404,1437.6,76.0,9.3,508.3,359.33,268.0
2,01/02/2022 2:00,11.9,421,1535.5,76.0,8.0,525.0,371.00,298.0
3,01/02/2022 3:00,12.2,425,1659.0,68.3,5.3,529.3,374.00,275.7
4,01/02/2022 4:00,11.9,430,1782.5,60.7,2.7,533.7,377.00,253.3
...,...,...,...,...,...,...,...,...,...
20973,23/06/2024 19:00,39.3,92,155.6,21.0,105.3,57.0,31.33,17.0
20974,23/06/2024 20:00,37.0,98,191.4,30.0,80.0,58.0,34.00,20.0
20975,23/06/2024 21:00,35.6,98,233.6,39.7,60.7,58.0,34.33,23.7
20976,23/06/2024 22:00,34.4,99,275.8,49.3,41.3,58.0,34.67,27.3


In [18]:
data = df[['so2']]

In [19]:
from sklearn.preprocessing import MinMaxScaler
values = data.values

values = values.astype('float32')

scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

In [21]:
scaled = scaled.reshape((20978,))

In [22]:
scaled.shape

(20978,)

In [23]:
def df_to_X_y(df, window_size, forecast_length):
    # df_as_np = df.to_numpy()
    X = []
    y = []
    
    for i in range(len(df) - window_size - forecast_length):
        row_x = [[a] for a in df[i:i+window_size]]
        
        X.append(row_x)
        # row_y = df_as_np[i+window_size]
        row_y = [b for b in df[i+window_size:i+window_size+forecast_length]]
        
        y.append(row_y)
        
    return np.array(X), np.array(y)

In [24]:
def train_test_split(X, y, test_size=0.2):
    split_idx = int(len(X) * (1 - test_size))
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    return X_train, X_test, y_train, y_test

In [25]:
WINDOW_WIDTH = 168
FORECAST_LENGTH = 168
X, y = df_to_X_y(scaled, WINDOW_WIDTH, FORECAST_LENGTH)


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

X_max = X_train.max()
X_min = X_train.min()

In [26]:
batchSize = 16
from keras.models import Sequential, save_model, load_model
from keras.layers import *
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError

In [27]:
normalizer = Normalization()
normalizer.adapt(X_train)
rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

model = Sequential()
# 1D Convolutional layers
model.add(Conv1D(filters=8, kernel_size=3, activation='relu', input_shape=X_train.shape[1:]))
# model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
# model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
# Flatten the output of the Conv layers
model.add(Flatten())
# model.add(LSTM(input_shape = X_train.shape[1:], units=86, return_sequences=True))
# model.add(LSTM(units=64, return_sequences=True))
# model.add(GRU(units=86, return_sequences=True))
# model.add(LSTM(units=128, return_sequences=True))
# model.add(GRU(units=100, return_sequences=True))
# model.add(LSTM(units=100, return_sequences=False))
# model.add(Dense(units=100,  activation='relu'))
# model.add(Dense(8, activation='relu'))

model.add(Dense(units=FORECAST_LENGTH, activation='linear'))

model.summary()

model.compile(optimizer=Adam(learning_rate=0.001), metrics=[RootMeanSquaredError()] , loss=MeanSquaredError())

history = model.fit(X_train, y_train, validation_data=(X_val,y_val), epochs=50, callbacks=[rlrop])

# score = model.evaluate(X_test, y_test, verbose = 1)

C:\Users\Hassan\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)                    │ (None, 166, 8)              │              32 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1328)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 168)                 │         223,272 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 223,304 (872.28 KB)

 Trainable params: 223,304 (872.28 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.0064 - root_mean_squared_error: 0.0767 - val_loss: 0.0016 - val_root_mean_squared_error: 0.0394 - learning_rate: 0.0010
Epoch 2/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0029 - root_mean_squared_error: 0.0538 - val_loss: 0.0018 - val_root_mean_squared_error: 0.0427 - learning_rate: 0.0010
Epoch 3/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0029 - root_mean_squared_error: 0.0535 - val_loss: 0.0019 - val_root_mean_squared_error: 0.0435 - learning_rate: 0.0010
Epoch 4/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0028 - root_mean_squared_error: 0.0525 - val_loss: 0.0015 - val_root_mean_squared_error: 0.0382 - learning_rate: 0.0010
Epoch 5/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0028 - root_mean_squared_error: 0.0528 - val_loss: 0.0018 - val_root_mean_squared_error: 0.0426 - learning_rate: 0.0010
Epoch 6/50
413/413 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0028 - root_mean_squared_err

In [28]:
forecast = model.predict(X_test)

130/130 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [29]:
poll = np.array(df["so2"])

meanop = poll.mean()
stdop = poll.std()

y_test_true = y_test*stdop + meanop
testPredict = forecast*stdop + meanop

In [30]:
from sklearn.metrics import mean_squared_error

In [31]:
rmse = np.sqrt(mean_squared_error(y_test_true, testPredict))
print("Test RMSE ="  ,rmse)

Test RMSE = 1.7281339


In [32]:
# Calculate MAPE
mape = np.mean(np.abs((y_test_true - testPredict) / y_test_true)) * 100
print("Test MAPE =", mape)

Test MAPE = 2.550477348268032


In [33]:
# Calculate accuracy as 1 - MAPE
accuracy = 100 - mape
print("Accuracy =", accuracy, "%")

Accuracy = 97.44952265173197 %


In [34]:
from sklearn.metrics import mean_absolute_error

# Calculate MAE
mae = mean_absolute_error(y_test_true, testPredict)
print("Test MAE =", mae)

Test MAE = 1.1748443


In [35]:
y_test_true.shape

(4129, 168)

In [36]:
testPredict.shape

(4129, 168)

In [37]:
import numpy as np

def calculate_segment_rmse(predictions, actuals, segment_length=1):
    num_segments = predictions.shape[1] // segment_length
    rmses = []
    
    for i in range(num_segments):
        start_idx = i * segment_length
        end_idx = start_idx + segment_length
        
        # Extract the segment for both predictions and actuals
        pred_segment = predictions[:, start_idx:end_idx]
        actual_segment = actuals[:, start_idx:end_idx]
        
        # Calculate RMSE for this segment
        segment_rmse = np.sqrt(mean_squared_error(actual_segment.flatten(), pred_segment.flatten()))
        rmses.append(segment_rmse)
    
    return rmses

In [38]:
segment_rmses = calculate_segment_rmse(testPredict, y_test_true)
for i, rmse in enumerate(segment_rmses):
    print(f"RMSE for segment {i + 1}: {rmse:.4f}")

RMSE for segment 1: 0.4049
RMSE for segment 2: 0.6550
RMSE for segment 3: 0.8732
RMSE for segment 4: 1.0374
RMSE for segment 5: 1.1659
RMSE for segment 6: 1.2532
RMSE for segment 7: 1.3142
RMSE for segment 8: 1.3485
RMSE for segment 9: 1.3791
RMSE for segment 10: 1.3965
RMSE for segment 11: 1.4117
RMSE for segment 12: 1.4260
RMSE for segment 13: 1.4404
RMSE for segment 14: 1.4510
RMSE for segment 15: 1.4608
RMSE for segment 16: 1.4690
RMSE for segment 17: 1.4782
RMSE for segment 18: 1.4913
RMSE for segment 19: 1.4943
RMSE for segment 20: 1.4991
RMSE for segment 21: 1.5084
RMSE for segment 22: 1.5071
RMSE for segment 23: 1.5136
RMSE for segment 24: 1.5158
RMSE for segment 25: 1.5303
RMSE for segment 26: 1.5499
RMSE for segment 27: 1.5762
RMSE for segment 28: 1.5970
RMSE for segment 29: 1.6178
RMSE for segment 30: 1.6329
RMSE for segment 31: 1.6527
RMSE for segment 32: 1.6551
RMSE for segment 33: 1.6625
RMSE for segment 34: 1.6670
RMSE for segment 35: 1.6716
RMSE for segment 36: 1.6762
R

In [39]:
forecast

array([[0.3066334 , 0.2813147 , 0.2528489 , ..., 0.20921755, 0.21632111,
        0.2198432 ],
       [0.2892909 , 0.26237142, 0.24014388, ..., 0.21525466, 0.22416669,
        0.22767617],
       [0.2642786 , 0.23813124, 0.21813536, ..., 0.2200374 , 0.23148546,
        0.21512215],
       ...,
       [0.06355341, 0.06920167, 0.072556  , ..., 0.04348197, 0.04945981,
        0.05511759],
       [0.0639976 , 0.06772599, 0.06454934, ..., 0.05039424, 0.05468717,
        0.05762226],
       [0.06153944, 0.05761218, 0.05529806, ..., 0.05332372, 0.0563986 ,
        0.05885588]], dtype=float32)

In [40]:
testPredict

array([[55.551857, 54.375824, 53.053616, ..., 51.026985, 51.356937,
        51.520535],
       [54.746315, 53.49593 , 52.46348 , ..., 51.3074  , 51.721355,
        51.88437 ],
       [53.584515, 52.369995, 51.441208, ..., 51.529556, 52.061306,
        51.301247],
       ...,
       [44.261024, 44.52338 , 44.679184, ..., 43.328724, 43.606388,
        43.869186],
       [44.281654, 44.454834, 44.307285, ..., 43.64979 , 43.849194,
        43.985527],
       [44.167477, 43.985058, 43.87757 , ..., 43.785866, 43.928688,
        44.042828]], dtype=float32)

In [41]:
y_test_true

array([[55.60104 , 54.76734 , 53.93364 , ..., 50.837036, 51.23007 ,
        51.63501 ],
       [54.76734 , 53.93364 , 53.099937, ..., 51.23007 , 51.63501 ,
        52.028038],
       [53.93364 , 53.099937, 51.46827 , ..., 51.63501 , 52.028038,
        50.837036],
       ...,
       [44.08406 , 44.131702, 44.16743 , ..., 42.97643 , 43.33373 ,
        43.691032],
       [44.131702, 44.16743 , 44.048332, ..., 43.33373 , 43.691032,
        44.131702],
       [44.16743 , 44.048332, 43.92923 , ..., 43.691032, 44.131702,
        44.560463]], dtype=float32)

In [42]:
model.save("so2_1dcnn.keras")